# 👤 Face Analysis: Age & Gender Prediction

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **comprehensive face analysis** using the FaceAnalyzer class:

- ✅ Detect faces with bounding boxes and landmarks
- ✅ Predict age and gender attributes
- ✅ Extract face embeddings for recognition
- ✅ Compare faces using similarity scores

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets/test_images', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["test_images/image0.jpg", "test_images/image1.jpg", "test_images/image2.jpg"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import matplotlib.pyplot as plt

import uniface
from uniface import FaceAnalyzer
from uniface.detection import RetinaFace
from uniface.recognition import ArcFace
from uniface.attribute import AgeGender
from uniface.visualization import draw_detections

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize FaceAnalyzer

The FaceAnalyzer combines **detection**, **recognition**, and **attribute prediction** in one class.

In [ ]:
analyzer = FaceAnalyzer(
    detector=RetinaFace(confidence_threshold=0.5),
    recognizer=ArcFace(),
    age_gender=AgeGender()
)

## 4️⃣ Analyze Faces in Multiple Images

In [ ]:
image_paths = [
    'assets/test_images/image0.jpg',
    'assets/test_images/image1.jpg',
    'assets/test_images/image2.jpg',
]

results = []

for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f'⚠ Error: Could not read {image_path}')
        continue

    faces = analyzer.analyze(image)
    print(f'\n📷 {image_path.split("/")[-1]}: Detected {len(faces)} face(s)')

    for i, face in enumerate(faces, 1):
        print(f'   Face {i}: {face.sex}, {face.age}y')

    vis_image = image.copy()
    bboxes = [f.bbox for f in faces]
    scores = [f.confidence for f in faces]
    landmarks = [f.landmarks for f in faces]
    draw_detections(image=vis_image, bboxes=bboxes, scores=scores, landmarks=landmarks, vis_threshold=0.5, fancy_bbox=True)

    results.append((image_path, cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB), faces))

## 5️⃣ Visualize Results with Attribute Info

In [ ]:
fig, axes = plt.subplots(2, len(results), figsize=(15, 8), gridspec_kw={'height_ratios': [4, 1]})

for idx, (path, vis_image, faces) in enumerate(results):
    axes[0, idx].imshow(vis_image)
    axes[0, idx].axis('off')

    axes[1, idx].axis('off')
    info_text = f'{len(faces)} face(s)\n'
    for i, face in enumerate(faces, 1):
        info_text += f'Face {i}: {face.sex}, {face.age}y\n'

    axes[1, idx].text(0.5, 0.5, info_text, ha='center', va='center', fontsize=11, family='monospace',
                      bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

plt.tight_layout()
plt.show()

## 6️⃣ Access All Face Attributes

Each Face object contains detection, recognition, and attribute data.

In [ ]:
_, _, faces = results[0]
if faces:
    face = faces[0]

    print('📋 Face Attributes:')
    print(f'   • Bounding box: {face.bbox.astype(int).tolist()}')
    print(f'   • Confidence: {face.confidence:.3f}')
    print(f'   • Landmarks shape: {face.landmarks.shape}')
    print(f'   • Age: {face.age} years')
    print(f'   • Gender: {face.sex}')
    print(f'   • Embedding shape: {face.embedding.shape}')
    print(f'   • Embedding dimension: {face.embedding.shape[1]}D')

## 7️⃣ Compare Face Similarity

Use face embeddings to compute similarity between detected faces.

In [ ]:
if len(results) >= 2:
    face1 = results[0][2][0]
    face2 = results[1][2][0]

    similarity = face1.compute_similarity(face2)
    is_same = similarity > 0.6
    
    print(f'🔄 Comparing faces from first two images:')
    print(f'   • Similarity: {similarity:.4f}')
    print(f'   • Same person: {"✅ Yes" if is_same else "❌ No"} (threshold=0.6)')

---

## 📝 Summary

| Attribute | Description |
|-----------|-------------|
| `face.bbox` | Bounding box `[x1, y1, x2, y2]` |
| `face.confidence` | Detection confidence (0-1) |
| `face.landmarks` | 5-point facial landmarks |
| `face.age` | Predicted age (years) |
| `face.sex` | Predicted gender ("Male"/"Female") |
| `face.embedding` | 512D face embedding for recognition |

### Methods

| Method | Description |
|--------|-------------|
| `analyzer.analyze(image)` | Full analysis (detect + recognize + attributes) |
| `face.compute_similarity(other)` | Cosine similarity between two faces |

---

## 🔗 Next Steps

- **Face Parsing**: Semantic segmentation → [06_face_parsing.ipynb](./06_face_parsing.ipynb)
- **Face Anonymization**: Blur faces for privacy → [07_face_anonymization.ipynb](./07_face_anonymization.ipynb)
- **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)